In [1]:
!pip install langchain-chroma langchain langchain-community langchain-huggingface langchain-openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 639.8 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.6 MB/s eta 0:00:00


In [2]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

In [3]:
embedding_model = HuggingFaceEmbeddings(
    model_name="AITeamVN/Vietnamese_Embedding",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [4]:
!unzip -o /content/bm_db_legal_test.zip

Archive:  /content/bm_db_legal_test.zip
   creating: bm_db_legal_test/
   creating: bm_db_legal_test/3c8c3cd3-331e-4a35-8463-51e6285c7833/
  inflating: bm_db_legal_test/3c8c3cd3-331e-4a35-8463-51e6285c7833/data_level0.bin  
  inflating: bm_db_legal_test/3c8c3cd3-331e-4a35-8463-51e6285c7833/header.bin  
  inflating: bm_db_legal_test/3c8c3cd3-331e-4a35-8463-51e6285c7833/length.bin  
  inflating: bm_db_legal_test/3c8c3cd3-331e-4a35-8463-51e6285c7833/link_lists.bin  
  inflating: bm_db_legal_test/chroma.sqlite3  


In [5]:
persist_dir = "./bm_db_legal_test"

law_db = Chroma(
    collection_name="law_docs",
    embedding_function=embedding_model,
    persist_directory=persist_dir
)

In [6]:
def retrieval_test(question):
  retriever = law_db.as_retriever(search_kwargs={"k": 5})
  query = question

  results = retriever.invoke(query)

  for r in results:
      print("\n--- Kết quả ---")
      print("Nội dung:", r.page_content)
      print("Metadata:", r.metadata)

In [78]:
system_prompt = """Bạn là một chuyên gia tư vấn pháp luật với 30 năm kinh nghiệm trong lĩnh vực này.
Nhiệm vụ của bạn là trả lời câu hỏi của người dùng về pháp luật của Việt Nam dựa trên thông tin được cung cấp.
Bạn phải đảm bảo trả lời câu hỏi một các chính xác, đầy đủ, dễ hiểu và phù hợp với các quy định pháp luật hiện hành.

**Yêu cầu trả lời:**

### 1. Phân tích câu hỏi:
- Hiểu rõ toàn diện về câu hỏi, bao gồm các cách diễn đạt tương tự, các từ đồng nghĩa và các biên thể ngữ nghĩa hoặc cách diễn đạt không trực tiếp.
- Xác định rõ các khía cạnh chính hoặc các điểm cần làm rõ từ câu hỏi

### 2. Cấu trúc câu trả lời:
- Mở đầu bằng một câu trả lời chính cho câu hỏi một cách tổng hợp tóm tắt, ngắn gọn, bao quát đầy đủ ý, đi thẳng vào trọng tâm vấn đề chính.
- Trả lời theo từng ý rõ ràng, mỗi ý tương ứng với một điểm hoặc khía cạnh cụ thể.
- Sử dụng các số thứ tự (1, 2, 3...) hoặc dấu đầu dòng để trình bày các ý của nội dung một cách logic .
- Lưu ý bắt buộc: Với **mỗi ý** trong câu trả lời, bạn **bắt buộc phải trích dẫn chính xác nguồn từ phần Metadata** của tài liệu tham khảo tương ứng khi dùng để trả lời ý đó.

### 3. **Trích dẫn nguồn:**
- Trích dẫn nuồn theo mẫu sau: (*Nguồn: Điều [Điều]: [Tên điều], Chương [Chương] - [Tên chương], văn bản [Loại văn bản]: [Chủ đề], số [Mã số], ban hành ngày [Ngày ban hành], tại [Nơi ban hành], bởi [Cơ quan ban hành], tài liệu gốc [Tên file tài liệu gốc]).
- Các ví dụ mẫu:
(*Nguồn: Điều 14, Chương II - Thẩm quyền ban hành nội dung văn bản quy phạm pháp luật, văn bản Luật: Ban hành văn bản quy phạm pháp luật, số 64/2025/QH15, ban hành ngày 19/02/2025 tại Hà Nội, bởi Quốc hội, tài liệu gốc L_64_2025_QH15.pdf)
(*Nguồn: Điều 47, Chương IV - Xây dựng ban hành văn bản quy phạm pháp luật của các cơ quan khác, văn bản Luật: Ban hành văn bản quy phạm pháp luật, số 64/2025/QH15, ban hành ngày 19/02/2025 tại Hà Nội, bởi Quốc hội, tài liệu gốc L_64_2025_QH15.pdf)
(*Nguồn: Điều 26, Chương III - Xây dựng ban hành văn bản quy phạm pháp luật của Quốc hội Ủy ban, văn bản Luật: Ban hành văn bản quy phạm pháp luật, số 64/2025/QH15, ban hành ngày 19/02/2025 tại Hà Nội, bởi Quốc hội, tài liệu gốc L_64_2025_QH15.pdf)
(*Nguồn: Điều 23, Chương III - Xây dựng ban hành văn bản quy phạm pháp luật của Quốc hội Ủy ban, văn bản Luật: Ban hành văn bản quy phạm pháp luật, số 64/2025/QH15, ban hành ngày 19/02/2025 tại Hà Nội, bởi Quốc hội, tài liệu gốc L_64_2025_QH15.pdf)
(*Nguồn: Điều 1, Chương I - Xây dựng, ban hành văn bản quy phạm pháp luật liên tịch, văn bản Luật: Ban hành văn bản quy phạm pháp luật, số 64/2025/QH15, ban hành ngày 19/02/2025 tại Hà Nội, bởi Quốc hội, tài liệu gốc L_64_2025_QH15.pdf)
- Bắt buộc phải trích dẫn theo đúng định dạng trong mẫu.
- Phải tìm và trích dẫn nguồn từ phần Metadata sử dụng để trả lời với mỗi ý, không được trả về các câu như: "tài liệu gốc không có thông tin đầy đủ", "tài liệu gốc",...
- Metadata được cung cấp ở cuối mỗi tài liệu, vì vậy sẽ luôn có nguồn.

### 4. Tuân thủ nội dung:
- Chỉ sử dụng thông tin từ nội dung được cung cấp trong phần 'Tài liệu tham khảo:' để trả lời câu hỏi, có thể suy diễn và suy luận từ thông tin. Nhưng tuyệt đối không thêm thông tin từ bên ngoài.

### 5. Xử lý trường hợp không có thông tin:
- Nêu không có thông tin phù hợp trong dữ liệu cung cấp, bắt buộc phải trả lời như sau: 'Tôi không tìm thấy thông tin trong tài liệu'.

### 6. Phong cách trình bày:
- Không quá dài, rõ ràng nhưng vẫn đảm bảo đủ độ chi tiết, đủ ý và thông tin.
- Chuyên nghiệp và chính xác theo ngôn ngữ pháp luật Việt Nam.
- Tránh sử dụng ngôn ngữ không trang trọng.

### 7. Đảm bảo chất lượng:
- Mọi câu trả lời cần được kiểm tra để đảm bảo tối đa tính chính xác và rõ ràng trước khi gửi đi.

Lưu ý quan trọng: Tất cả các những yêu cầu trên chỉ dùng để hướng dẫn những quy định khi bạn trả lời câu hỏi, khi trả lời bạn chỉ cần đảm bảo các yêu cầu trên, bắt buộc khi trả lời chỉ thực hiện trả lời theo yêu cầu và tuyệt đối không được tạo ra các yêu cầu ### phía trên.
"""

In [66]:
metadata_label_map = {
    "co_quan_ban_hanh": "Cơ quan ban hành",
    "ten_chuong": "Tên chương",
    "chu_de": "Chủ đề",
    "noi_ban_hanh": "Nơi ban hành",
    "ngay_ban_hanh": "Ngày ban hành",
    "ten_dieu": "Tên điều",
    "dieu": "Điều",
    "ma_so": "Mã số",
    "chuong": "Chương",
    "loai_van_ban": "Loại văn bản",
    "source": "Nguồn"
}

In [67]:
def create_prompt_template(system_prompt):
  prompt_template = ChatPromptTemplate.from_messages([
      ("system", system_prompt),
      ("human", """
      Dưới đây là câu hỏi bạn cần trả lời:
      {question}

      Tài liệu tham khảo:
      {context}
      """)
  ])

  return prompt_template

In [68]:
def format_metadata_vietnamese(metadata):
    parts = []
    for key, value in metadata.items():
        label = metadata_label_map.get(key, key)
        parts.append(f"{label}: {value}")
    return "(Metadata: " + "; ".join(parts) + ")"

In [93]:
def get_context(query, retriever):
    documents = retriever.invoke(query)

    contexts = []
    for i, doc in enumerate(documents, 1):
        content = doc.page_content
        metadata_formated = format_metadata_vietnamese(doc.metadata)
        contexts.append(f"Tài liệu {i}:\n{content}\n{metadata_formated}")

    context = "\n\n".join(contexts)

    return context

In [94]:
def add_metadata_to_content(doc):
    meta = format_metadata_vietnamese(doc.metadata)
    doc.page_content += f"\n{meta}"
    return doc

In [107]:
def get_response(prompt_template, llm, question, context):
    chain = prompt_template | llm
    answer = chain.invoke({"question": question, "context": context})

    return answer.content

In [108]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")

openai_api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    model = 'gpt-4o',
    openai_api_key = openai_api_key,
    temperature=0
)

In [210]:
question = "Một địa phương muốn ban hành chính sách thu hút đầu tư riêng, có thể ban hành văn bản như thế nào?"
retriever = law_db.as_retriever(search_kwargs={"k": 5})
context = get_context(question, retriever)
prompt_template = create_prompt_template(system_prompt)
chain = prompt_template | llm
answer = get_response(prompt_template, llm, question, context)

In [211]:
print(chain)

first=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Bạn là một chuyên gia tư vấn pháp luật với 30 năm kinh nghiệm trong lĩnh vực này.\nNhiệm vụ của bạn là trả lời câu hỏi của người dùng về pháp luật của Việt Nam dựa trên thông tin được cung cấp.\nBạn phải đảm bảo trả lời câu hỏi một các chính xác, đầy đủ, dễ hiểu và phù hợp với các quy định pháp luật hiện hành.\n\n**Yêu cầu trả lời:**\n\n### 1. Phân tích câu hỏi:\n- Hiểu rõ toàn diện về câu hỏi, bao gồm các cách diễn đạt tương tự, các từ đồng nghĩa và các biên thể ngữ nghĩa hoặc cách diễn đạt không trực tiếp.\n- Xác định rõ các khía cạnh chính hoặc các điểm cần làm rõ từ câu hỏi\n\n### 2. Cấu trúc câu trả lời:\n- Mở đầu bằng một câu trả lời chính cho câu hỏi một cách tổng hợp tóm tắt, ngắn gọn, bao quát đầy đủ ý, đi thẳng vào trọng tâm vấn đề chính.\n- Trả lờ

In [212]:
print(context)

Tài liệu 1:
1. Hội đồng nhân dân cấp tỉnh ban hành nghị quyết để quy định:
a) Chi tiết điều, khoản, điểm và các nội dung khác được giao trong văn bản quy phạm pháp luật của cơ quan
nhà nước cấp trên;
b) Chính sách, biện pháp nhằm bảo đảm thi hành Hiến pháp, luật, văn bản quy phạm pháp luật của cơ quan
nhà nước cấp trên;
c) Biện pháp nhằm phát triển kinh tế - xã hội, ngân sách, quốc phòng, an ninh ở địa phương; biện pháp khác
có tính chất đặc thù phù hợp với điều kiện phát triển kinh tế - xã hội của địa phương; thực hiện nhiệm vụ,
quyền hạn được phân cấp;
d) Thực hiện thí điểm các chính sách theo quy định của Luật Tổ chức chính quyền địa phương.
2. Ủy ban nhân dân cấp tỉnh ban hành quyết định để quy định:
a) Chi tiết điều, khoản, điểm và các nội dung khác được giao trong văn bản quy phạm pháp luật của cơ quan
nhà nước cấp trên;
b) Biện pháp thi hành Hiến pháp, luật, văn bản quy phạm pháp luật của cơ quan nhà nước cấp trên, nghị
(Metadata: Ngày ban hành: 19/02/2025; Loại văn bản: luật; C

In [213]:
print(prompt_template)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Bạn là một chuyên gia tư vấn pháp luật với 30 năm kinh nghiệm trong lĩnh vực này.\nNhiệm vụ của bạn là trả lời câu hỏi của người dùng về pháp luật của Việt Nam dựa trên thông tin được cung cấp.\nBạn phải đảm bảo trả lời câu hỏi một các chính xác, đầy đủ, dễ hiểu và phù hợp với các quy định pháp luật hiện hành.\n\n**Yêu cầu trả lời:**\n\n### 1. Phân tích câu hỏi:\n- Hiểu rõ toàn diện về câu hỏi, bao gồm các cách diễn đạt tương tự, các từ đồng nghĩa và các biên thể ngữ nghĩa hoặc cách diễn đạt không trực tiếp.\n- Xác định rõ các khía cạnh chính hoặc các điểm cần làm rõ từ câu hỏi\n\n### 2. Cấu trúc câu trả lời:\n- Mở đầu bằng một câu trả lời chính cho câu hỏi một cách tổng hợp tóm tắt, ngắn gọn, bao quát đầy đủ ý, đi thẳng vào trọng tâm vấn đề chính.\n- Trả lời theo từng ý rõ ràng, mỗi ý

In [214]:
print(answer)

Để một địa phương ban hành chính sách thu hút đầu tư riêng, có thể thực hiện thông qua các văn bản quy phạm pháp luật sau:

1. **Hội đồng nhân dân cấp tỉnh** có thể ban hành nghị quyết để quy định các biện pháp nhằm phát triển kinh tế - xã hội, bao gồm các chính sách thu hút đầu tư phù hợp với điều kiện phát triển kinh tế - xã hội của địa phương. Điều này bao gồm cả việc thực hiện thí điểm các chính sách theo quy định của Luật Tổ chức chính quyền địa phương (*Nguồn: Điều 21, Chương II - Thẩm quyền ban hành nội dung văn bản quy phạm pháp luật, văn bản Luật: Ban hành văn bản quy phạm pháp luật, số 64/2025/QH15, ban hành ngày 19/02/2025 tại Hà Nội, bởi Quốc hội, tài liệu gốc L_64_2025_QH15.pdf*).

2. **Ủy ban nhân dân cấp tỉnh** có thể ban hành quyết định để quy định chi tiết các biện pháp thi hành Hiến pháp, luật, và các văn bản quy phạm pháp luật của cơ quan nhà nước cấp trên, bao gồm các chính sách thu hút đầu tư (*Nguồn: Điều 21, Chương II - Thẩm quyền ban hành nội dung văn bản quy ph

In [215]:
def test_questions(llm, retriever, prompt_template_func, get_context_func, get_response_func):
    questions = {
        "question1": "Theo Luật số 64/2025/QH15, văn bản quy phạm pháp luật được ban hành bằng ngôn ngữ gì?",
        "question2": "Trong trường hợp xảy ra tham nhũng, tiêu cực trong quá trình xây dựng pháp luật, ai sẽ bị xử lý? Và theo căn cứ nào?",
        "question3": "Thời hạn gửi hồ sơ dự án luật cho cơ quan thẩm tra trước kỳ họp Quốc hội là ít nhất bao nhiêu ngày?",
        "question4": "Luật số 64/2025/QH15 được ban hành vào thời gian nào và bởi cơ quan nào?",
        "question5": "Một người đứng đầu đã phân công cấp phó phụ trách công tác xây dựng pháp luật, nhưng cấp phó vi phạm. Người đứng đầu có bị trách nhiệm không?",
        "question6": "Người từ đủ bao nhiêu tuổi phải chịu trách nhiệm hình sự?",
        "question7": "Điều 68 có quy định chi tiết mức phạt hành chính cho hành vi ban hành văn bản trái luật không?",
        "question8": "Có trường hợp nào mà cá nhân vi phạm có thể được miễn hoặc giảm trách nhiệm không?",
        "question9": "Phản biện xã hội và tham vấn chính sách khác nhau như thế nào?",
        "question10": "Một địa phương muốn ban hành chính sách thu hút đầu tư riêng, có thể ban hành văn bản như thế nào?"
    }

    for key, question in questions.items():
        context = get_context_func(question, retriever)
        prompt_template = prompt_template_func(system_prompt)
        answer = get_response_func(prompt_template, llm, question, context)
        print(f"{key}: {question}")
        print("→ Trả lời:", answer)
        print("="*80)

In [216]:
test_questions(
    llm=llm,
    retriever=law_db.as_retriever(search_kwargs={"k": 5}),
    prompt_template_func=create_prompt_template,
    get_context_func=get_context,
    get_response_func=get_response
)

question1: Theo Luật số 64/2025/QH15, văn bản quy phạm pháp luật được ban hành bằng ngôn ngữ gì?
→ Trả lời: Văn bản quy phạm pháp luật được ban hành bằng ngôn ngữ tiếng Việt. Ngôn ngữ này phải đảm bảo tính chính xác, phổ thông, thống nhất, diễn đạt rõ ràng và dễ hiểu. Ngoài ra, văn bản quy phạm pháp luật có thể được dịch ra tiếng dân tộc thiểu số và tiếng nước ngoài, nhưng bản dịch chỉ có giá trị tham khảo (*Nguồn: Điều 7: Ngôn ngữ thể thức kỹ thuật trình bày và dịch văn bản quy phạm pháp luật ra tiếng dân tộc, Chương I - Những quy định chung, văn bản Luật: Ban hành văn bản quy phạm pháp luật, số 64/2025/QH15, ban hành ngày 19/02/2025 tại Hà Nội, bởi Quốc hội, tài liệu gốc L_64_2025_QH15.pdf).
question2: Trong trường hợp xảy ra tham nhũng, tiêu cực trong quá trình xây dựng pháp luật, ai sẽ bị xử lý? Và theo căn cứ nào?
→ Trả lời: Trong trường hợp xảy ra tham nhũng, tiêu cực trong quá trình xây dựng pháp luật, các đối tượng sau đây sẽ bị xử lý:

1. **Cơ quan, tổ chức, người đứng đầu và 